In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
%cd D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data

D:\ky4\MachineLearning\MLP\Toxic-comment-classification\clean_data


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
def write_to_file(data, filename):
    with open(filename, 'w') as f:
        for line in data:
            f.write(line + '\n')

def read_from_file(filename):
    with open(filename, 'r') as f:
        return np.array(f.read().splitlines())

# Dataframe Train

In [5]:
train = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\train.csv").fillna('')
test = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\test.csv").fillna('')
submission = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\sample_submission.csv")

In [6]:
y_train_origin = pd.read_csv("D:\\ky4\\MachineLearning\\Project\\test_labels.csv")
classes = y_train_origin.columns.values[1:]
classes[0] 

'toxic'

# Transform tool


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, strip_accents='unicode', use_idf=True, smooth_idf=True, sublinear_tf=True)

In [8]:
X_train_origin = read_from_file("D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\data_train_cleaned_vanilla.txt")

In [9]:
X_test_origin = read_from_file("D:\\ky4\\MachineLearning\\MLP\\Toxic-comment-classification\\clean_data\\data_test_cleaned_vanilla.txt")

In [10]:
train.loc[:, 'comment_text'] = X_train_origin
train = train[:1000]
test.loc[:, 'comment_text'] = X_test_origin
test = test[:1000]

In [11]:
X_tr = train['comment_text'].values
y = train[classes].values
X_te = test['comment_text'].values
# X_te_vec = vec.transform(X_te)

# code có vẻ lặp nhưng cần thiết
train_trans = vec.fit_transform(train.comment_text)
test_trans = vec.transform(test.comment_text)

# K-fold


In [12]:
from sklearn.model_selection import KFold
num_folds = 10
scores = []
oof_predict = np.zeros((train.shape[0], 6))

predict = np.zeros((test.shape[0], 6))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)


In [17]:
from sklearn.calibration import CalibratedClassifierCV, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import GridSearchCV

def trainGrid (X_train_trans, X_test_trans, y_train, y_test, train_trans, test_trans, num_col, label):
    # X_train, X_test, y_train, y_test = train_test_split(train, train[label], test_size=0.3, random_state= 50)

    # selector = SelectKBest(chi2, k=int(0.2*X_train_trans.shape[1]))  # Select top 100 features (adjust k as needed)
    # X_train_trans = selector.fit_transform(X_train_trans, y_train[:, num_col])
    # # print("Shape:" + str(X_train_trans.shape))
    # X_test_trans = selector.transform(X_test_trans)

    def pr(y_i, y):
        p = X_train_trans[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)
    def get_mdl(y):
        y = pd.Series(y)
        y = y.values
        r = np.log(pr(1,y) / pr(0,y))
        return y,r
    
    print(y_train[:, num_col].shape)
    y,r = get_mdl(y_train[:, num_col])
    x_nb = X_train_trans.multiply(r)
    
    param_grid = {
        'C': [0.1, 1, 10, 100, 1000],'loss': ['hinge', 'squared_hinge']}
    grid = GridSearchCV(LinearSVC(),param_grid,refit = True, cv= 3, verbose=2)
    grid.fit(x_nb, y)

    model = CalibratedClassifierCV(LinearSVC('l2', loss = grid.best_params_["loss"],C = grid.best_params_["C"]), cv = 3)
  
    # train_trans = selector.fit_transform(train_trans, train[label])

    print(train_trans.shape, train[label].shape)

    fitted_model = model.fit(train_trans.multiply(r), train[label])
    # test_trans = selector.transform(test_trans)


    return fitted_model, test_trans, r


In [18]:
for train_index, test_index in kf.split(X_tr):
    kfold_X_train, kfold_X_valid = X_tr[train_index], X_tr[test_index]
    kfold_y_train, kfold_y_valid = y[train_index], y[test_index]

    # Vectorize text data
    kfold_X_train_vec = vec.fit_transform(kfold_X_train)
    kfold_X_valid_vec = vec.transform(kfold_X_valid)

    # Initialize an empty list to store models for each label
    models = []
    
    # Train a separate Naive Bayes model for each label
    for i, label in enumerate(classes):
        model, res_train, r = trainGrid(X_train_trans=kfold_X_train_vec, X_test_trans=kfold_X_valid_vec, y_train=kfold_y_train, y_test=kfold_y_valid, train_trans=train_trans, test_trans=test_trans, num_col=i, label=label)
        models.append([model,res_train,r])

    # model predict cái test nhân thêm r thì nó là naive bayes !

    predict += np.array([model[0].predict_proba(model[1].multiply(model[2]))[:, 1] for model in models]).T / num_folds  
    oof_predict[test_index] = np.array([model[0].predict_proba(kfold_X_valid_vec)[:, 1] for model in models]).T


(900,)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ..................................C=0.1, loss=hinge; total time=   0.0s
[CV] END ..................................C=0.1, loss=hinge; total time=   0.0s
[CV] END ..................................C=0.1, loss=hinge; total time=   0.0s
[CV] END ..........................C=0.1, loss=squared_hinge; total time=   0.0s
[CV] END ..........................C=0.1, loss=squared_hinge; total time=   0.0s
[CV] END ..........................C=0.1, loss=squared_hinge; total time=   0.0s
[CV] END ....................................C=1, loss=hinge; total time=   0.0s
[CV] END ....................................C=1, loss=hinge; total time=   0.0s
[CV] END ....................................C=1, loss=hinge; total time=   0.0s
[CV] END ............................C=1, loss=squared_hinge; total time=   0.0s
[CV] END ............................C=1, loss=squared_hinge; total time=   0.0s
[CV] END ............................C=1,

d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The 

[CV] END ..................................C=100, loss=hinge; total time=   0.0s
[CV] END ..................................C=100, loss=hinge; total time=   0.0s
[CV] END ..................................C=100, loss=hinge; total time=   0.0s
[CV] END ..........................C=100, loss=squared_hinge; total time=   0.0s
[CV] END ..........................C=100, loss=squared_hinge; total time=   0.0s


d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning

[CV] END ..........................C=100, loss=squared_hinge; total time=   0.0s
[CV] END .................................C=1000, loss=hinge; total time=   0.0s
[CV] END .................................C=1000, loss=hinge; total time=   0.0s
[CV] END .................................C=1000, loss=hinge; total time=   0.0s
[CV] END .........................C=1000, loss=squared_hinge; total time=   0.0s


d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
d:\PMLT\Miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning

[CV] END .........................C=1000, loss=squared_hinge; total time=   0.0s
[CV] END .........................C=1000, loss=squared_hinge; total time=   0.0s
(1000, 5085) (1000,)


ValueError: inconsistent shapes

In [ ]:
print(predict[:2,:])
print('Done')